In [9]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_columns', 500)

MARKET = "eth_rlp_usdc"
# MARKET = "eth_cbbtc_usdt"
# MARKET = "base_cbbtc_usdc_full"


df = pd.read_csv(f"/Users/yegortrussov/Documents/ml/lending_protocols/dataset_collection/data/markets_enriched/{MARKET}.csv")

with open("/Users/yegortrussov/Documents/ml/lending_protocols/dataset_collection/data/common/markets_meta.json", 'r') as f:
    all_markets_meta = json.load(f)
    market_address = df["market_address"].unique()[0]
    market_meta = all_markets_meta[market_address]
    collateral_asset_address = market_meta["collateral_asset_address"]
with open("/Users/yegortrussov/Documents/ml/lending_protocols/dataset_collection/data/common/assets_meta.json", 'r') as f:
    all_assets_meta = json.load(f)

market_meta.keys(), all_assets_meta.keys()

/var/folders/hj/pbs977kd43s6n1l9z3mxrj200000gn/T/ipykernel_49744/960219989.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"/Users/yegortrussov/Documents/ml/lending_protocols/dataset_collection/data/markets_enriched/{MARKET}.csv")


(dict_keys(['address', 'lltv', 'oracle_address', 'creation_datetime', 'network', 'loan_asset_address', 'loan_asset_symbol', 'loan_asset_decimals', 'collateral_asset_address', 'collateral_asset_symbol', 'collateral_asset_decimals', 'rate_at_target', 'irm_curve']),
 dict_keys(['0xdAC17F958D2ee523a2206206994597C13D831ec7', '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', '0x4956b52aE2fF65D74CA2d61207523288e4528f96', '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0xcbB7C0000aB88B473b1f5aFd9ef808440eed33Bf', '0x7CF9DEC92ca9FD46f8d86e7798B72624Bc116C05']))

In [10]:
from tqdm import tqdm
df["hash_short"] = df["hash"].str[:10]

def label_transaction_sequences(df, time_threshold_seconds=300):
    df = df.sort_values(['user_address', 'timestamp']).copy()
    df['event_sequence_type'] = df['event_type'].copy()
    inx = 0
    
    for i in tqdm(range(1, len(df))):
        # Check if same user
        if df.iloc[i]['user_address'] != df.iloc[i-1]['user_address']:
            inx = 0
            continue
        inx += 1
        addr = df.iloc[i]['user_address']
            
        # Check time difference
        time_diff = df.iloc[i]['timestamp'] - df.iloc[i-1]['timestamp']
        if time_diff > time_threshold_seconds:
            continue
        
        
        # Check for collateral + borrow sequence
        if (df.iloc[i-1]['event_type'] == 'collateral_add' and 
            df.iloc[i]['event_type'] == 'borrow_more' and inx > 1):
            df.at[df.index[i-1], 'event_sequence_type'] = 'borrow_more_w_collateral'
            df.at[df.index[i], 'event_sequence_type'] = 'borrow_more_w_collateral'
        if (df.iloc[i-1]['event_type'] == 'repay_full' and 
            df.iloc[i]['event_type'] == 'position_close'):
            df.at[df.index[i-1], 'event_sequence_type'] = 'position_close'
            df.at[df.index[i], 'event_sequence_type'] = 'position_close'
        if (df.iloc[i-1]['event_type'] == 'collateral_add' and 
            df.iloc[i]['event_type'] == 'borrow_more' and inx == 1):
            df.at[df.index[i-1], 'event_sequence_type'] = 'position_open'
            df.at[df.index[i], 'event_sequence_type'] = 'position_open'
    
    return df

df_labeled = label_transaction_sequences(df, 60*10)

# df_labeled = pd.read_csv("./base_cbbtc_usdc_labeled.csv")

100%|██████████| 56184/56184 [00:11<00:00, 4790.36it/s]


In [11]:

def create_event_chains(df, max_sequence_length=30):
    """
    Creates event chains for each user with financial metrics.
    """
    def compress_sequence(events):
        compressed = []
        count = 1
        
        for i, event in enumerate(events):
            if i == 0:
                compressed.append(event)
            elif event == compressed[-1]:
                count += 1
            else:
                if count > 1:
                    compressed[-1] = f"{compressed[-1]}({count})"
                compressed.append(event)
                count = 1
        
        if count > 1:
            compressed[-1] = f"{compressed[-1]}({count})"
        
        return compressed

    df_sorted = df.sort_values(['user_address', 'timestamp']).copy().drop_duplicates("hash")
    
    chains = []
    
    for user, user_df in df_sorted.groupby('user_address'):
        events = user_df['event_sequence_type'].tolist()
        timestamps = user_df['timestamp'].tolist()
        
        # Extract financial metrics arrays
        collateral_after = user_df['collateral_after'].values
        debt_after = user_df['debt_after'].values
        ltv_after = user_df['ltv_after'].values
        ltv_before = user_df['ltv_before'].values
        collateral_value_after = user_df['collateral_value_after'].values
        
        # Create compressed chain
        compressed_events = []
        compressed_timestamps = []
        compressed_indices = []
        
        last_event = None
        for idx, (event, ts) in enumerate(zip(events, timestamps)):
            if event != last_event:
                compressed_events.append(event)
                compressed_timestamps.append(ts)
                compressed_indices.append(idx)
                last_event = event
        
        # Process segments
        chain_parts = []
        current_timestamps = []
        current_indices = []
        chain_index = 0
        
        for i, (event, ts, idx) in enumerate(zip(compressed_events, compressed_timestamps, compressed_indices)):
            chain_parts.append(event)
            current_timestamps.append(ts)
            current_indices.append(idx)
            
            if event == "position_close" or i == (len(compressed_events) - 1):
                if not current_indices:
                    chain_parts = []
                    current_timestamps = []
                    current_indices = []
                    continue
                
                # Get financial data for this segment
                segment_indices = current_indices
                segment_collateral = collateral_after[segment_indices]
                segment_debt = debt_after[segment_indices]
                segment_ltv = ltv_after[segment_indices]
                segment_before_ltv = ltv_before[segment_indices]
                segment_collateral_value = collateral_value_after[segment_indices]
                
                # Calculate metrics
                initial_debt = segment_debt[0] if len(segment_debt) > 0 else 0
                max_debt = segment_debt.max() if len(segment_debt) > 0 else 0
                final_debt = segment_debt[-1] if len(segment_debt) > 0 else 0
                
                initial_collateral = segment_collateral[0] if len(segment_collateral) > 0 else 0
                max_collateral = segment_collateral.max() if len(segment_collateral) > 0 else 0
                final_collateral = segment_collateral[-1] if len(segment_collateral) > 0 else 0
                
                initial_collateral_value = segment_collateral_value[0] if len(segment_collateral_value) > 0 else 0
                max_collateral_value = segment_collateral_value.max() if len(segment_collateral_value) > 0 else 0
                
                initial_ltv = max(segment_ltv[0], segment_before_ltv[0]) if len(segment_ltv) > 0 else 0
                max_ltv = max(segment_ltv.max(), segment_before_ltv.max()) if len(segment_ltv) > 0 else 0
                final_ltv = max(segment_ltv[-1], segment_before_ltv[-1]) if len(segment_ltv) > 0 else 0
                
                # Chain string
                chain = " → ".join(chain_parts[:max_sequence_length])
                
                if chain_parts[0].startswith("loan"):
                    chain_parts = []
                    current_timestamps = []
                    current_indices = []
                    continue
                
                chains.append({
                    'user_address': user,
                    'chain_id': f"{user[:10]}_{chain_index}",
                    'event_chain': chain,
                    'chain_length': len(current_indices),
                    'position_length_days': (current_timestamps[-1] - current_timestamps[0]) / (24*3600),
                    'total_events': len(chain_parts),
                    
                    # Financial metrics
                    'initial_debt': initial_debt,
                    'max_debt': max_debt,
                    
                    'initial_collateral': initial_collateral,
                    'max_collateral': max_collateral,
                    'final_collateral': final_collateral,
                    
                    'initial_collateral_value': initial_collateral_value,
                    'max_collateral_value': max_collateral_value,
                    
                    'initial_ltv': initial_ltv,
                    'max_ltv': max_ltv,
                    'ltv_diff': max_ltv - initial_ltv,
                    
                    'avg_debt': segment_debt.mean() if len(segment_debt) > 0 else 0,
                    'avg_ltv': segment_ltv.mean() if len(segment_ltv) > 0 else 0,
                    'ltv_volatility': segment_ltv.std() if len(segment_ltv) > 1 else 0,
                    
                    # Timestamps
                    'first_timestamp': current_timestamps[0],
                    'last_timestamp': current_timestamps[-1],
                    'first_event': chain_parts[0],
                    'last_event': chain_parts[-1],
                    
                    # Risk metrics
                    'was_liquidated': 1 if 'liquidation' in chain.lower() else 0,
                })
                
                chain_index += 1
                chain_parts = []
                current_timestamps = []
                current_indices = []
    
    chains_df = pd.DataFrame(chains)
    return chains_df

chains_df = create_event_chains(df_labeled[df_labeled["vault_flg"] == False])
chains_df.head(3)

def simplify_chain_risk_level(chain):
    if pd.isna(chain):
        return chain
    
    events = chain.split(' → ')
    simplified = []
    
    for event in events:
        # Keep these as-is
        if event in ['position_open', 'position_close', 'liquidation', 'borrow_more_w_collateral', ]:
            simplified.append(event)
        # Risk decreasing events
        elif event in ['repay_full', 'repay_partial', 'loan_position_withdraw', 
                      'collateral_withdraw']:
            simplified.append('position_risk_decrease')
        # Risk increasing events  
        elif event in ['borrow_more', 'collateral_add',
                      'loan_position_supply']:
            simplified.append('position_risk_increase')
        # Position adjustments
        elif event == 'position_adjust':
            simplified.append('position_adjust')
        else:
            simplified.append('other')
    
    # Remove consecutive duplicates
    final_events = []
    for event in simplified:
        if not final_events or event != final_events[-1]:
            final_events.append(event)
    
    return ' → '.join(final_events)

def add_simplified_chain_column(df):
    df = df.copy()
    df['event_chain_collapsed'] = df['event_chain'].apply(simplify_chain_risk_level)
    return df

chains_df = add_simplified_chain_column(chains_df)
chains_df.head(2)

,user_address,chain_id,event_chain,chain_length,position_length_days,total_events,initial_debt,max_debt,initial_collateral,max_collateral,final_collateral,initial_collateral_value,max_collateral_value,initial_ltv,max_ltv,ltv_diff,avg_debt,avg_ltv,ltv_volatility,first_timestamp,last_timestamp,first_event,last_event,was_liquidated,event_chain_collapsed
0,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_0,position_open → position_close,2,0.351806,2,40793.268000,40793.268000,42679.0,42679.0,0.0,49507.64,49507.64,0.824115,0.824115,0.0,20395.176455,0.412058,0.412058,1742239427,1742269823,position_open,position_close,0,position_open → position_close
1,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_1,position_open → position_close,2,0.481250,2,81680.509294,81680.509294,85235.0,85235.0,0.0,99724.95,99724.95,0.819225,0.819225,0.0,40834.657279,0.409613,0.409613,1742661995,1742703575,position_open,position_close,0,position_open → position_close


In [12]:
df["user_address"].nunique()

691

In [13]:
chains_df.to_csv(f"/Users/yegortrussov/Documents/ml/lending_protocols/dataset_collection/data/markets_chains/{MARKET}.csv", index=False)
df_labeled.to_csv(f"/Users/yegortrussov/Documents/ml/lending_protocols/dataset_collection/data/markets_labelled/{MARKET}.csv", index=False)


In [14]:
def filter_by_user_time_range(df_a, df_b):
    df_b_ranges = df_b[['user_address', 'first_timestamp', 'last_timestamp']].drop_duplicates()
    
    # Merge with ranges
    df_merged = df_a.merge(df_b_ranges, on='user_address', how='left')
    
    # Filter
    filtered = df_merged[
        (df_merged['timestamp'] >= df_merged['first_timestamp']) & 
        (df_merged['timestamp'] <= df_merged['last_timestamp'])
    ].drop(columns=['first_timestamp', 'last_timestamp'])
    
    return filtered
def leave_valid_chains(chains_df, prefix="", len_lim=10, leave_first_k=None, contains_event=None):
    df_prep = chains_df[chains_df["event_chain"].str.startswith(prefix) & (chains_df["total_events"] < len_lim)]
    if contains_event:
        df_prep = df_prep[df_prep["event_chain"].str.contains(contains_event)]
    if leave_first_k is not None:
        df_prep['event_chain'] = df_prep['event_chain'].str.split('→').str[:leave_first_k].str.join('→').str.strip()
    return df_prep

filtered_events = filter_by_user_time_range(
    df,
    leave_valid_chains(
        chains_df,
        prefix="position_open",
        len_lim=15,
        leave_first_k=15,
        contains_event="position_close",
    )
)

In [15]:
filtered_events.shape, df.shape
filtered_events[filtered_events['user_address'] == '0x000Aee61aee8aA46422A2f29Ebefc57684A9Ee81']["event_type"]

Series([], Name: event_type, dtype: object)

In [16]:
chains_df

,user_address,chain_id,event_chain,chain_length,position_length_days,total_events,initial_debt,max_debt,initial_collateral,max_collateral,final_collateral,initial_collateral_value,max_collateral_value,initial_ltv,max_ltv,ltv_diff,avg_debt,avg_ltv,ltv_volatility,first_timestamp,last_timestamp,first_event,last_event,was_liquidated,event_chain_collapsed
0,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_0,position_open → position_close,2,0.351806,2,40793.268000,40793.268000,42679.000000,42679.000000,0.0,49507.640000,49507.640000,0.824115,0.824115,0.0000,20395.176455,0.412058,0.412058,1742239427,1742269823,position_open,position_close,0,position_open → position_close
1,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_1,position_open → position_close,2,0.481250,2,81680.509294,81680.509294,85235.000000,85235.000000,0.0,99724.950000,99724.950000,0.819225,0.819225,0.0000,40834.657279,0.409613,0.409613,1742661995,1742703575,position_open,position_close,0,position_open → position_close
2,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_2,position_open → position_close,2,0.040278,2,95484.029526,95484.029526,98702.000000,98702.000000,0.0,116468.360000,116468.360000,0.819869,0.819869,0.0000,47735.764558,0.409935,0.409935,1747301843,1747305323,position_open,position_close,0,position_open → position_close
3,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_3,position_open → position_close,2,0.038611,2,63779.461165,63779.461165,65878.000000,65878.000000,0.0,77736.040000,77736.040000,0.820565,0.820565,0.0000,31883.147697,0.410283,0.410283,1747646891,1747650227,position_open,position_close,0,position_open → position_close
4,0x000001ac4e512d670c34feDf6c71cE2F49fb160a,0x000001ac_4,position_open → position_close,2,0.047778,2,73491.542873,73491.542873,75784.000000,75784.000000,0.0,90182.960000,90182.960000,0.815086,0.815086,0.0000,36738.739391,0.407543,0.407543,1747905287,1747909415,position_open,position_close,0,position_open → position_close
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,0xfC67418ff4fFb7F39769587bFD425D750B9f0663,0xfC67418f_0,collateral_add → collateral_withdraw → positio...,3,91.201806,3,0.000000,0.000000,200.000000,200.000000,0.0,228.000000,228.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1736515523,1744395359,collateral_add,position_close,0,position_risk_increase → position_risk_decreas...
614,0xfa3EF2Ea5F719fdE7A291009212401bFE140a551,0xfa3EF2Ea_0,position_open → borrow_more_w_collateral → rep...,4,137.753750,4,259975.820000,542949.501000,263505.667366,546805.692739,0.0,321476.914186,667102.945141,0.808767,0.813967,0.0052,284092.252866,0.595067,0.344269,1753372811,1765274735,position_open,position_close,0,position_open → borrow_more_w_collateral → pos...
615,0xfaE44b36F3a4966015515191A1adDe6B2e2Fe286,0xfaE44b36_0,collateral_add → position_close,2,61.234167,2,0.000000,0.000000,10.000000,10.000000,0.0,11.400000,11.400000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1736430707,1741721339,collateral_add,position_close,0,position_risk_increase → position_close
616,0xfe08d06A422992ab8eb3dBcD1475f39c8d595ca5,0xfe08d06A_0,position_open → collateral_add → position_close,3,4.965139,3,43699.711279,43699.711279,44485.424579,48485.424579,0.0,54272.217987,59152.217987,0.805361,0.805361,0.0000,29113.671881,0.514760,0.365000,1753776599,1754205587,position_open,position_close,0,position_open → position_risk_increase → posit...
